In [9]:
from metrics import dice
import nibabel as nib
import os
import pickle
import numpy as np

In [6]:
class Args():
    def __init__(self):
        self.dir_project = r'..'
        self.dir_lf = r'D:\Data\cs-8395-dl'
        self.path_kfold = r'D:\Projects\cs-8395-dl\Assignment_3_segmentation\partition\kfold_5.bin'
        self.folderPartition = 'train_test_org'
        self.folderData = r'assignment3\Training'
        self.batchSize = 12
        self.axis = 'axial'
        self.lossWeight = [0.3, 0.7]
        self.load_from = r'D:/Data/cs-8395-dl/model/2020-03-23-03-22-40/2020-03-23-03-22-40_dpn68_fold-1_best.pt'
        self.dir_output = r'D:\Data\cs-8395-dl\output\assignment3\segmentation_cc_thresh_axial_prob_no_mask_organ'
args = Args()

In [7]:
dir_data = os.path.join(args.dir_lf,args.folderData) #os.path.join(DIR_LF,'assignment1_data')
with open(os.path.join(args.dir_project,'partition',args.folderPartition,'Training.txt')) as  f:
    file_ids_all = [id.strip() for id in f.readlines()]

In [11]:
kf = pickle.load(open(args.path_kfold,'rb'))
ind_train, ind_valid = list(kf.split(file_ids_all))[0]
file_ids = np.array(file_ids_all)[ind_valid]

In [23]:
np.array(file_ids_all)[ind_train]

array(['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008',
       '0021', '0022', '0023', '0024', '0025', '0027', '0029', '0030',
       '0031', '0032', '0033', '0035', '0036', '0037', '0039', '0040'],
      dtype='<U4')

In [24]:
np.array(file_ids_all)[ind_valid]

array(['0009', '0010', '0026', '0028', '0034', '0038'], dtype='<U4')

In [18]:
dice_coeff = []
for fl_id in file_ids:
    path_gt = os.path.join(dir_data,'label','label{}.nii.gz'.format(fl_id))
    gt = nib.load(path_gt).get_fdata()
    gt[gt!=1]=0
    path_out = os.path.join(args.dir_output,'label{}.nii.gz'.format(fl_id))
    out = nib.load(path_out).get_fdata()
    dice_coeff.append(dice(gt,out))

In [25]:
for fl_id,d in zip(file_ids,dice_coeff):
    print(fl_id,'{:.2f}'.format(d))

0009 0.69
0010 0.90
0026 0.96
0028 0.96
0034 0.97
0038 0.94


In [21]:
np.mean(dice_coeff), np.median(dice_coeff), np.std(dice_coeff)

(0.9035715805289071, 0.9490568276149373, 0.09605564932382006)